In [1]:
pip install pyspark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 3.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
## Structured Streaming 

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# SparkSession 생성
spark = SparkSession.builder \
    .appName("KafkaStructuredStreaming") \
    .config("spark.sql.streaming.schemaInference", "true") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .getOrCreate()

# Kafka에서 데이터 읽기
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "new_test_topic") \
    .option("startingOffsets", "earliest") \
    .load()

# Kafka에서 읽어온 데이터에서 value 컬럼을 문자열로 변환
value_df = kafka_df.selectExpr("CAST(value AS STRING)")

# 데이터 처리
processed_df = value_df.withColumn("processed_value", col("value"))

# 콘솔에 출력 (데이터를 콘솔에 보여주기 위한 코드)
query_console = processed_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# 데이터 메모리에 저장 (결과를 테이블에 저장해서 쿼리로 확인 가능)
query_memory = processed_df.writeStream \
    .queryName("kafka_data") \
    .outputMode("append") \
    .format("memory") \
    .start()

# 스트리밍 종료 대기
query_console.awaitTermination()

StreamingQueryException: [STREAM_FAILED] Query [id = 949794d3-50ac-42a7-a99a-079d44943f61, runId = f5a9f359-7f8a-49f1-b2bf-dd187be04f5d] terminated with exception: org.apache.kafka.common.errors.TimeoutException: Call(callName=describeTopics, deadlineMs=1725539823997, tries=1, nextAllowedTryMs=1725539824101) timed out at 1725539824001 after 1 attempt(s)

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# SparkSession 생성
spark = SparkSession.builder \
    .appName("KafkaStructuredStreamingWithAggregates") \
    .config("spark.sql.streaming.schemaInference", "true") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .getOrCreate()

# Kafka에서 데이터 읽기
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "new_test_topic") \
    .option("startingOffsets", "earliest") \
    .load()

# Kafka에서 읽어온 데이터에서 value 컬럼을 문자열로 변환
value_df = kafka_df.selectExpr("CAST(value AS STRING)")

# 데이터 처리: "processed_value" 필드에 대해 집계 (count)
# (여기서는 예시로, value 값을 기준으로 그룹화한 후 빈도(count) 계산)
aggregated_df = value_df.withColumn("processed_value", col("value")) \
    .groupBy("processed_value").count()

# 콘솔에 출력 (데이터의 집계 결과를 콘솔에 보여주기 위한 코드)
query_console = aggregated_df.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

# 스트리밍 종료 대기
query_console.awaitTermination()

StreamingQueryException: [STREAM_FAILED] Query [id = 7835fdc9-d3ae-47eb-9dbd-a7a68cda53f3, runId = a4868b09-909c-4391-a135-cbbc431dca7b] terminated with exception: org.apache.kafka.common.errors.TimeoutException: Call(callName=describeTopics, deadlineMs=1725540945289, tries=1, nextAllowedTryMs=1725540945390) timed out at 1725540945290 after 1 attempt(s)